# 🚦 Traffic Manager AI Platform (V2)
**Hybrid Engine**: YOLO11m (Vehicles) + YOLOv5 (License Plates)

### Instructions:
1. **Settings**: Enable **GPU** and **Internet** in Kaggle settings sidebar.
2. **Upload**: Upload your custom models (`mhiot-vehicle...` etc.) if you have them. If not, the script downloads YOLO11m automatically.
3. **Config**: Update `SOCKETIO_SERVER_URL` in the code below to your Ngrok URL.
4. **Run**: Run all cells.

In [ ]:
# 📦 Install Dependencies
!pip install ultralytics python-socketio[client] websocket-client opencv-python-headless pillow --quiet
print("✅ Dependencies Installed")

In [ ]:
# 📥 Pre-download YOLO11m Model (To avoid hanging during init)
print("⏳ Downloading YOLO11m model... (Please wait)")
!wget -nc https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt
print("✅ Model ready")

In [ ]:
import cv2
import numpy as np
import socketio
import base64
import time
import threading
import queue
import io
import torch
import math
import os
import sys
from PIL import Image
from ultralytics import YOLO

# ---------------- CONFIGURATION ----------------
# ⚠️ UPDATE THIS URL TO YOUR NGROK URL:
SOCKETIO_SERVER_URL = 'wss://unadjourned-darlene-subcarinated.ngrok-free.dev' 
CONFIDENCE_THRESHOLD = 0.5

# Model Paths
VEHICLE_MODEL_PATH = 'yolo11m.pt'               # YOLO11m (User requested)
# VEHICLE_MODEL_PATH = 'mhiot-vehicle-best-new.pt' # Custom model option
TRAFFIC_LIGHT_MODEL_PATH = 'mhiot-dentinhieu-best-new.pt'
LP_DETECTOR_PATH = 'LP_detector.pt'
LP_OCR_PATH = 'LP_ocr.pt'

# Vehicle Classes (COCO indices or names)
VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

# Feature Flags
ENABLE_VEHICLE = True
ENABLE_TRAFFIC_LIGHT = False
ENABLE_LICENSE_PLATE = False

# ----------------- GLOBALS -----------------
sio = socketio.Client(reconnection=True, ssl_verify=False, logger=False, engineio_logger=False)
camera_queues = {}
camera_threads = {}
running = True
connected = False

# Models
vehicle_model = None
traffic_light_model = None
lp_detect_model = None
lp_ocr_model = None

# ----------------- UTILS (LICENSE PLATE) -----------------
def changeContrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    return cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

def compute_skew(src_img):
    if len(src_img.shape) == 3: h, w, _ = src_img.shape
    else: h, w = src_img.shape
    img = cv2.medianBlur(src_img, 3)
    edges = cv2.Canny(img, 30, 100, apertureSize=3, L2gradient=True)
    lines = cv2.HoughLinesP(edges, 1, math.pi/180, 30, minLineLength=w/1.5, maxLineGap=h/3.0)
    if lines is None: return 0
    
    angle = 0.0
    cnt = 0
    for x1, y1, x2, y2 in lines[:, 0]:
        ang = np.arctan2(y2-y1, x2-x1)
        if math.fabs(ang) <= 30:
            angle += ang
            cnt += 1
    return (angle / cnt)*180/math.pi if cnt > 0 else 0

def deskew(src_img):
    return rotate_image(src_img, compute_skew(src_img))

def read_plate_yolov5(model, im):
    results = model(im)
    bb_list = results.pandas().xyxy[0].values.tolist()
    if len(bb_list) < 7: return "unknown"
    
    bb_list.sort(key=lambda x: x[0])
    ys = [b[1] for b in bb_list]
    if max(ys) - min(ys) > im.shape[0] * 0.4:
        y_mean = sum(ys) / len(ys)
        line1 = sorted([b for b in bb_list if b[1] < y_mean], key=lambda x: x[0])
        line2 = sorted([b for b in bb_list if b[1] >= y_mean], key=lambda x: x[0])
        text = "".join([str(b[-1]) for b in line1]) + "-" + "".join([str(b[-1]) for b in line2])
        return text
    else:
        return "".join([str(b[-1]) for b in bb_list])

# ----------------- MODEL LOADING -----------------
def load_models():
    global vehicle_model, traffic_light_model, lp_detect_model, lp_ocr_model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🚀 Loading models on {device}...")

    # 1. Vehicle (YOLOv8/11)
    if ENABLE_VEHICLE:
        try:
            print(f"Loading Vehicle: {VEHICLE_MODEL_PATH}")
            vehicle_model = YOLO(VEHICLE_MODEL_PATH)
            vehicle_model.to(device)
            print("✅ Vehicle Model Loaded (v8/11)")
        except Exception as e: print(f"❌ Vehicle Load Error: {e}")

    # 2. Traffic Light (YOLOv8/11)
    if ENABLE_TRAFFIC_LIGHT:
        try:
            print(f"Loading Traffic Light: {TRAFFIC_LIGHT_MODEL_PATH}")
            traffic_light_model = YOLO(TRAFFIC_LIGHT_MODEL_PATH)
            traffic_light_model.to(device)
            print("✅ Traffic Light Model Loaded (v8/11)")
        except Exception as e: print(f"❌ TL Load Error: {e}")

    # 3. License Plate (YOLOv5)
    if ENABLE_LICENSE_PLATE:
        try:
            print(f"Loading LP Detector (v5): {LP_DETECTOR_PATH}")
            sys.path.insert(0, os.getcwd())
            lp_detect_model = torch.hub.load('ultralytics/yolov5', 'custom', path=LP_DETECTOR_PATH, force_reload=True)
            lp_detect_model.to(device)
            lp_detect_model.conf = 0.3
            
            print(f"Loading LP OCR (v5): {LP_OCR_PATH}")
            lp_ocr_model = torch.hub.load('ultralytics/yolov5', 'custom', path=LP_OCR_PATH, force_reload=True)
            lp_ocr_model.to(device)
            lp_ocr_model.conf = 0.5
            print("✅ License Plate Models Loaded (v5)")
        except Exception as e: print(f"❌ LP Load Error: {e}")

# ----------------- INFERENCE -----------------
def run_v8_inference(model, frame, classes=None):
    if model is None: return []
    results = []
    h, w = frame.shape[:2]
    try:
        res = model(frame, verbose=False, conf=CONFIDENCE_THRESHOLD)
        for r in res:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                class_name = model.names[cls_id]
                if classes and class_name not in classes: continue
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                results.append({
                    'class': class_name,
                    'confidence': float(box.conf[0]),
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h}
                })
    except Exception as e: print(f"Inference v8 error: {e}")
    return results

def run_lp_inference(frame):
    if lp_detect_model is None or lp_ocr_model is None: return []
    results = []
    h, w = frame.shape[:2]
    try:
        preds = lp_detect_model(frame, size=640)
        df = preds.pandas().xyxy[0]
        for _, row in df.iterrows():
            if row['confidence'] < 0.3: continue
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            crop = frame[y1:y2, x1:x2]
            if crop.size == 0: continue
            
            text = "unknown"
            try:
                text = read_plate_yolov5(lp_ocr_model, crop)
                if text == "unknown":
                    text = read_plate_yolov5(lp_ocr_model, deskew(crop))
            except: pass
            
            if text != "unknown":
                results.append({
                    'type': 'license_plate',
                    'text': text,
                    'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h},
                    'confidence': row['confidence']
                })
    except Exception as e: print(f"LP Inference error: {e}")
    return results

# ----------------- MAIN LOOP -----------------
def process_camera(camera_id):
    global running
    print(f"📹 Cam {camera_id} loop started")
    while running:
        try:
            data = camera_queues[camera_id].get(timeout=0.1)
            frame, cam_id, img_id, created_at = data
            start = time.time()
            all_dets = []
            
            if ENABLE_VEHICLE:
                dets = run_v8_inference(vehicle_model, frame, classes=VEHICLE_CLASSES)
                for d in dets:
                    d['type'] = 'vehicle'
                    all_dets.append(d)
            
            if ENABLE_TRAFFIC_LIGHT:
                dets = run_v8_inference(traffic_light_model, frame)
                for d in dets: 
                    d['type'] = 'traffic_light'
                    all_dets.append(d)
                    
            if ENABLE_LICENSE_PLATE:
                all_dets.extend(run_lp_inference(frame))
            
            ms = (time.time() - start) * 1000
            if sio.connected and all_dets:
                sio.emit('car_detected', {
                    'camera_id': cam_id, 'image_id': img_id,
                    'detections': all_dets, 'inference_time': ms,
                    'created_at': created_at
                })
                print(f"  ✅ Cam {cam_id[-4:]}: {len(all_dets)} objs ({ms:.0f}ms)")
        except queue.Empty: continue
        except Exception: pass

# ----------------- EVENTS -----------------
@sio.event
def connect():
    global connected
    connected = True
    print(f"✅ Connected to {SOCKETIO_SERVER_URL}")
    sio.emit("join_all_camera")

@sio.event
def disconnect():
    global connected
    connected = False
    print("⚠️ Disconnected")

@sio.on('image')
def on_image(data):
    try:
        img_data = data.get('image') or data.get('buffer')
        if isinstance(img_data, dict): img_data = bytes(img_data.get('data', []))
        if isinstance(img_data, str): img_data = base64.b64decode(img_data)
        frame = cv2.imdecode(np.frombuffer(img_data, np.uint8), cv2.IMREAD_COLOR)
        if frame is None: return
        
        cid = data['cameraId']
        if cid not in camera_queues:
            camera_queues[cid] = queue.Queue(maxsize=2)
            threading.Thread(target=process_camera, args=(cid,), daemon=True).start()
        
        q = camera_queues[cid]
        if q.full(): 
            try: q.get_nowait()
            except: pass
        q.put((frame, cid, data['imageId'], data.get('created_at', 0)))
    except: pass

def maintain():
    while running:
        if not sio.connected:
            print(f"🔄 Connecting to {SOCKETIO_SERVER_URL}...")
            try: 
                sio.connect(SOCKETIO_SERVER_URL, transports=['websocket'])
                print("✅ Socket connected!")
            except Exception as e: 
                print(f"❌ Connection failed: {e}")
                time.sleep(5)
        time.sleep(1)

if __name__ == "__main__":
    load_models()
    threading.Thread(target=maintain, daemon=True).start()
    try:
        while running: time.sleep(10)
    except: running = False